In [2]:
!pip install bertopic
!pip install arxiv

import locale
locale.getpreferredencoding = lambda: "UTF-8"

import pandas as pd
import numpy as np
from bertopic import BERTopic
import arxiv
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/data/df_experiment')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df.head(5)

,Unnamed: 0,entry_id,updated,published,title,summary,primary_category,categories,authors
0,0,http://arxiv.org/abs/2305.11154v1,2023.42,2023.42,non linear operator valued elliptic flows with...,differential equations on spaces of operators ...,ph,['mp'],"['jean bernard bru', 'nathan metraud']"
1,1,http://arxiv.org/abs/2305.11103v1,2023.42,2023.42,blockwise inversion and algorithms for inverti...,using the blockwise matrix inversion inversion...,na,"['na', 'na', 'mp']",['r thiru senthil']
2,2,http://arxiv.org/abs/2305.11054v1,2023.42,2023.42,ising systems measures on the sphere and zonoids,we give an interpretation of a class of discre...,ap,"['ap', 'mp', 'oc']","['andrea braides', 'antonin chambolle']"
3,3,http://arxiv.org/abs/2210.09458v2,2023.42,2022.83,mobility edge for levy matrices,levy matrices are symmetric random matrices wh...,pr,"['pr', 'mp']","['amol aggarwal', 'charles bordenave', 'patric..."
4,4,http://arxiv.org/abs/2205.08765v2,2023.42,2022.42,necessary and sufficient conditions for one di...,this paper deals with necessary and sufficient...,ca,"['ca', 'ft', 'mp']",['pavol quittner']


In [4]:
abstracts = df.summary.to_list()
len(abstracts)

4462

In [5]:
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose = True) # We need the probabilities to visualize
topics, probs = topic_model.fit_transform(abstracts)

Batches:   0%|          | 0/140 [00:00<?, ?it/s]

2023-05-25 15:25:23,347 - BERTopic - Transformed documents to Embeddings
2023-05-25 15:25:49,161 - BERTopic - Reduced dimensionality
2023-05-25 15:25:49,963 - BERTopic - Clustered reduced embeddings


In [6]:
topic_model.get_topic_info().head(5)

,Topic,Count,Name
0,-1,1829,-1_the_of_we_and
1,0,172,0_theories_gauge_theory_the
2,1,158,1_fractional_solutions_problem_solution
3,2,146,2_stokes_the_navier_solutions
4,3,141,3_kahler_complex_manifold_manifolds


In [7]:
# Get the most frequent topics
topic_freq = topic_model.get_topic_freq()
outliers = topic_freq['Count'][topic_freq['Topic']==-1].iloc[0]
print(f"{outliers} documents have not been classified")
print(f"The other {topic_freq['Count'].sum() - outliers} documents are {topic_freq['Topic'].shape[0]-1} topics")

1829 documents have not been classified
The other 2633 documents are 57 topics


In [8]:
topic_freq.head(5)

,Topic,Count
0,-1,1829
1,0,172
2,1,158
3,2,146
4,3,141


In [9]:
print(f"There are {topic_freq['Count'].iloc[1]} documents that are talking about topic ID {topic_freq['Topic'].iloc[1]}")

There are 172 documents that are talking about topic ID 0


In [10]:
for i in range(topic_freq['Topic'].shape[0]):
  print(f'topic{i}:')
  print(topic_model.get_topic(topic_freq['Topic'].iloc[i]))
  print('')
  print('')

topic0:
[('the', 0.015713086812285778), ('of', 0.01556118923153465), ('we', 0.013277423855797848), ('and', 0.013159821727756139), ('in', 0.0127243482062428), ('to', 0.01229448494633788), ('is', 0.01126359294704973), ('for', 0.011262871370357821), ('on', 0.010772391454306033), ('that', 0.01049443901182131)]


topic1:
[('theories', 0.01955573904066662), ('gauge', 0.018340469821947467), ('theory', 0.016537040917361724), ('the', 0.015627928410598495), ('symmetry', 0.014322499310704352), ('of', 0.014279056297167058), ('in', 0.01368739460048531), ('field', 0.013528091182440256), ('and', 0.01349423160128119), ('to', 0.012499384371420228)]


topic2:
[('fractional', 0.02960047892247741), ('solutions', 0.024488314622883874), ('problem', 0.0191194022246631), ('solution', 0.017966436415206437), ('boundary', 0.017386677019350646), ('existence', 0.016942679265982523), ('the', 0.016344859525143904), ('in', 0.016110821175417678), ('for', 0.01583146361124335), ('and', 0.015021829674411758)]


topic3:
[

In [11]:
topic_model.visualize_topics()

In [12]:
topic_model.visualize_documents(abstracts)

In [13]:
probs.shape

(4462, 57)

In [14]:
topic_model.visualize_distribution(probs[57])

In [15]:
topic_model.visualize_hierarchy()

In [16]:
topic_model.visualize_barchart(top_n_topics=5)

In [17]:
topic_model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

In [18]:
topic_model.visualize_term_rank()

In [19]:
topic_model.save("/content/drive/MyDrive/june_bertopic_model")	

In [20]:
topic_model.update_topics(abstracts, n_gram_range=(2, 4))

In [21]:
topic_model.get_topic(3)

[('of the', 0.00650038603714234),
 ('on the', 0.005218727479295161),
 ('kahler manifold', 0.0046652290956292995),
 ('compact kahler', 0.0042276198851160535),
 ('existence of', 0.004095391396993689),
 ('we show', 0.004088655991108963),
 ('kahler manifolds', 0.004051469056569552),
 ('in the', 0.004000173190798264),
 ('show that', 0.003966674862387553),
 ('to the', 0.0038981418959174144)]

In [22]:
topic_model.get_representative_docs(3)

['we prove two rigidity results on holomorphic isometries into homogeneous kahler manifolds the first shows that a kahler ricci soliton induced by the homogeneous metric of the kahler product of a special flag manifold ie a flag of classical type or integral type with a bounded homogeneous domain is trivial ie kahler einstein in the second one we prove that i a flat space is not relative to the kahler product of a special flag manifold with a homogeneous bounded domain ii a special flag manifold is not relative to the kahler product of a flat space with a homogeneous bounded domain and iii a homogeneous bounded domain is not relative to the kahler product of a flat space with a special flag manifold our theorems strongly extend the results in     and ',
 'let be an dimensional polarized manifold let be a smooth hypersurface defined by a holomorphic section of  in this paper we study the existence of a complete scalar flat kahler metric on on the assumption that has a constant positive 

# - the Naivest Possible Arxiv Recommender Using BERTopic -

In [51]:
topic_model = BERTopic.load("/content/drive/MyDrive/june_bertopic_model")	

In [52]:
# make a function that eats a list of arxiv ids and spits out the list of abstracts of the corresponding ids

In [53]:
df_train =df.copy()
user_input_ids = ['0905.0486', 'math/0006187', '2106.07444', '1402.0490', 
                  '1512.08942', '1603.09235', 'math/0510265', 'math/0505056', 
                  'math/0604379', '2209.02568']
df_daily =df.copy()

In [54]:
# using a sample of 10 papers from the training data because I can't import arxiv library on colab
# if arxiv library worked I would have made a function that (eats a list of arxiv ids and spits out 
# the list of abstracts of the corresponding ids) and applied it to the original list_user inside the above cell
list_user = df.sample(10).summary.to_list()

In [55]:
results =  arxiv.Search(id_list = user_input_ids).results()
list_user = []
for result in results :
  list_user.append(result.summary)

In [56]:
daily_topics = topic_model.transform(abstracts)
daily_topics = daily_topics[0]
topic_freq = topic_model.get_topic_freq()

Batches:   0%|          | 0/140 [00:00<?, ?it/s]

2023-05-25 16:14:28,729 - BERTopic - Reduced dimensionality
2023-05-25 16:14:29,883 - BERTopic - Calculated probabilities with HDBSCAN
2023-05-25 16:14:29,884 - BERTopic - Predicted clusters


In [57]:
# 1.make a list where the ith entry is the sub-dataframe of df_library with topic i
# and then make a list where the ith entry is the 
daily_dfs = []
daily_arrs = []
for i in range(topic_freq['Topic'].shape[0]) :
  idxs = [k for k,topic in enumerate(daily_topics) if topic == i]
  if idxs :
    daily_dfs.append(df_daily.iloc[idxs])
    daily_arrs.append(topic_model.embedding_model.embed(daily_dfs[i].summary.to_list()))
  else :
    daily_dfs.append(pd.DataFrame())
    daily_arrs.append(np.array([]))
  


In [58]:
daily_dfs[0]

,Unnamed: 0,entry_id,updated,published,title,summary,primary_category,categories,authors
26,26,http://arxiv.org/abs/2301.04538v2,2023.42,2023.08,galilean fermions classical and quantum aspects,we study the classical and quantum properties ...,th,['mp'],['aditya sharma']
27,27,http://arxiv.org/abs/2304.09506v2,2023.42,2023.33,nonlinear realisation approach to topologicall...,we develop a nonlinear realisation approach to...,th,['mp'],"['sergei m kuzenko', 'jake c stirling']"
37,37,http://arxiv.org/abs/2305.09650v1,2023.42,2023.42,quantum geometrodynamics revived ii hilbert sp...,this paper represents the second in a series o...,qc,['mp'],"['thorsten lang', 'susanne schander']"
42,42,http://arxiv.org/abs/2305.09451v1,2023.42,2023.42,the celestial chiral algebra of self dual grav...,we consider the twistor description of classic...,th,['mp'],"['roland bittleston', 'simon heuveline', 'davi..."
73,73,http://arxiv.org/abs/2211.11162v2,2023.42,2022.92,covariant superspace approaches to supergravity,we provide a unified description of the three ...,th,['mp'],"['s m kuzenko', 'e s n raptakis', 'g tartaglin..."
...,...,...,...,...,...,...,...,...,...
4142,676,http://arxiv.org/abs/1808.04894v2,2023.25,2018.67,transgression of d branes,closed strings can be seen either as one dimen...,ph,"['dg', 'mp']","['severin bunk', 'konrad waldorf']"
4143,677,http://arxiv.org/abs/2207.11799v2,2023.25,2022.58,geometric t duality buscher rules in general t...,the classical buscher rules describe t duality...,dg,"['dg', 'at', 'mp']",['konrad waldorf']
4240,776,http://arxiv.org/abs/2209.09250v3,2023.25,2022.75,an embedding formalism for cfts in general sta...,we present a generalisation of the embedding s...,th,['dg'],"['enrico parisini', 'kostas skenderis', 'benja..."
4264,800,http://arxiv.org/abs/2210.08628v2,2023.25,2022.83,electric magnetic duality in a class of compac...,we study electric magnetic duality in compacti...,th,"['ag', 'dg']","['james halverson', 'benjamin sung', 'jiahua t..."


In [59]:
daily_arrs[0]

array([[-0.09735502, -0.05551469, -0.02901678, ...,  0.00611959,
         0.01746424,  0.03043462],
       [-0.04344057, -0.1059176 ,  0.00925388, ..., -0.02918508,
         0.0040202 ,  0.01062678],
       [-0.07758626,  0.02730094, -0.02353329, ...,  0.00809414,
        -0.03178598, -0.02032119],
       ...,
       [-0.07021944, -0.05164965,  0.02280099, ..., -0.00270366,
        -0.00165346, -0.04203246],
       [-0.07835192, -0.01855108,  0.06653189, ..., -0.06005681,
        -0.01485121, -0.02253856],
       [-0.07309654,  0.0803986 ,  0.08116895, ..., -0.00157206,
        -0.02248685, -0.03637279]], dtype=float32)

In [60]:
daily_arrs[0].shape

(172, 384)

In [61]:
# cosine similarity function
def norm(vec):
  return np.sqrt((vec**2).sum())
def cos_sim(vec1,vec2):
  return (vec1 * vec2).sum()/(norm(vec1)*norm(vec2))

In [62]:
# experiment with cosine similarity function
cos_sim(np.array([1,0]), np.array([0,1]))

0.0

In [63]:
# convert the list of abstracts of user input papers into vectors using
# sBERT inside the trained BERTopic model(i.e. topic_model) 
user_vecs = topic_model.embedding_model.embed(list_user)

# inference the topics that the user input papers belong to
user_topics = topic_model.transform(list_user)[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-05-25 16:14:37,797 - BERTopic - Reduced dimensionality
2023-05-25 16:14:37,816 - BERTopic - Calculated probabilities with HDBSCAN
2023-05-25 16:14:37,817 - BERTopic - Predicted clusters


In [64]:
user_vecs.shape

(10, 384)

In [65]:
outputs =[]
for i in range(len(list_user)):
  # this part is unnecessary but I'm doing this to simplify notations
  # vector representing ith paper in the user input 
  user_vec = user_vecs[i].reshape(1,-1)
  # the topic index that ith paper in the user input belongs to
  user_topic = user_topics[i]

  # compute cosine similarity scores between ith paper in the user input and 
  # the daily papers(the papers we will recommend from) with the topic == user_topic
  similarity_scores = [cos_sim(user_vec, vec) for vec in daily_arrs[user_topic]]
  # check whether daily papers with topic == user_topic is empty or not
  if similarity_scores :
    # if daily papers with topic == user_topic is not empty
    # append the title of the daily paper with highest similarity score with ith paper in the user input
    argmax = max(enumerate(similarity_scores), key=lambda x: x[1])[0]
    outputs.append(daily_dfs[user_topic].iloc[argmax].summary)
  else : 
    # if daily papers with topic == user_topic is empty 
    outputs.append('NA')

In [66]:
outputs

['we give new proofs that khovanov homology detects the figure eight knot and the cinquefoils and that homfly homology detects and each of the pretzel knots for all but the figure eight these mostly follow the same lines as in previous work the key difference is that in honor of tom mrowkas th birthday the arguments here use instanton floer homology rather than knot floer homology',
 'let be a complete dimensional almost kahler manifold first part of this article we construct some identities of various laplacians generalized hodge and serre dualities a generalized hard lefschetz duality and a lefschetz decomposition all on the space of on pure bidegree in the second part as some applications of those identities we establish some vanishing theorems on the spaces of harmonic forms on under some growth assumptions on the kaher form  we also give some estimates to sharpen the vanishing theorems in two specific cases at last of the article as an application we study the topology and geometr

# To do's
instead of restricting cosine_similarity calculations to papers with the most relevant topic, restrict to top 3 most relevant topics.

# concern  
what if user input papers don't belong to any of the topics. What if the 'true' best paper to recommend is in unclassified paper list.